In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import time

In [17]:
df=pd.read_csv('vehicle.csv')
df

,Color,Type,Origin,Stolen
0,Red,Sports,Domestic,Yes
1,Red,Sports,Domestic,No
2,Red,Sports,Domestic,Yes
3,Yellow,Sports,Domestic,Yes
4,Yellow,Sports,Imported,Yes
5,Yellow,Suv,Imported,No
6,Yellow,Suv,Imported,Yes
7,Yellow,Suv,Domestic,No
8,Red,Suv,Imported,No
9,Red,Sports,Imported,Yes


In [28]:
def CategTraining(X_train,y_train,outCol):
    trainCount=y_train[outCol[0]].value_counts().to_dict()
    col1=list(X_train.columns)
    outputList=y_train[outCol[0]].value_counts().keys().tolist()
    trainDict=dict([(key, []) for key in col1])
    for i in range(0,len(col1)):
        inputList=X_train[col1[i]].value_counts().keys().tolist()
        proxy=dict([(key, dict([(keys, []) for keys in outputList])) for key in inputList])
        trainDict[col1[i]]=proxy
    for i in trainDict.keys():
        for j in trainDict[i].keys():
            for k in trainDict[i][j].keys():
                num=(X_train.loc[(X_train[i]==j) &  (y_train[outCol[0]]==k)]).shape[0]
                den=trainCount[k]
                prob=num/den
                trainDict[i][j][k]=prob
    return trainDict

def CategTesting(outCol,trainDict,y_train,y_test,X_test):
    trainProb=(y_train[outCol[0]].value_counts()/y_train.shape[0]).to_dict()
    outputList=y_train[outCol[0]].value_counts().keys().tolist()
    testDict1=dict([(key,dict([(keys, []) for keys in outputList])) for key in y_test.index])
    for i in testDict1.keys():
        for j in testDict1[i].keys():
            prob=1
            l=0
            for k in trainDict.keys():
                prob=trainDict[k][X_test.loc[i][l]][j]*prob
                l=l+1
            testDict1[i][j]=prob*trainProb[j]
    return testDict1

def CategPredict(testDict):
    predict=[]
    for i in testDict.keys():
        maxi=0
        l=''
        for j in testDict[i].keys():
            if(testDict[i][j]>maxi):
                maxi=testDict[i][j]
                l=j
        predict.append(l)
    return predict

In [4]:
print('Enter the value of k')
k=int(input())
size=int(df.shape[0])
groupCount=int(size/k)
r=0
predict=[]
while(r<size):
    testingData=df.iloc[r:r+groupCount]
    trainingData=df.iloc[r+groupCount:size]
    if(r!=0):
        trainingData=trainingData.append(df.iloc[0:r])
    r=r+groupCount
    X_train=trainingData[['Color','Type','Origin']]
    y_train=trainingData[['Stolen']]
    X_test=testingData[['Color','Type','Origin']]
    y_test=testingData[['Stolen']]
    outCol=list(y_train.columns)
    trainDict=CategTraining(X_train,y_train,outCol)
    testDict=CategTesting(outCol,trainDict,y_train,y_test,X_test)
    proxy=CategPredict(testDict,y_test)
    for i in proxy:
        predict.append(i)
print(predict)
accuracy=((df.loc[df['Stolen']==predict]).shape[0]/df.shape[0])*100
print(accuracy)

Enter the value of k
5
['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes']
80.0


In [60]:
from sklearn.model_selection import KFold
X=df[['Color','Type','Origin']]
y=df[['Stolen']]
kf=KFold(n_splits=5,shuffle=True)
predict=[None]*df.shape[0]
for train_index,test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    outCol=list(y_train.columns)
    trainDict=CategTraining(X_train,y_train,outCol)
    testDict=CategTesting(outCol,trainDict,y_train,y_test,X_test)
    print(testDict)
    proxy=CategPredict(testDict)
    for i in range(0,len(proxy)):
        predict[test_index[i]]=proxy[i]
    print(predict)
accuracy=((df.loc[df['Stolen']==predict]).shape[0]/df.shape[0])*100
print(accuracy)

TRAIN: [0 1 2 3 4 5 7 8] TEST: [6 9]
{6: {'No': 0.09375, 'Yes': 0.0}, 9: {'No': 0.03125, 'Yes': 0.0625}}
[None, None, None, None, None, None, 'No', None, None, 'Yes']
TRAIN: [0 1 2 5 6 7 8 9] TEST: [3 4]
{3: {'No': 0.03125, 'Yes': 0.046875}, 4: {'No': 0.03125, 'Yes': 0.046875}}
[None, None, None, 'Yes', 'Yes', None, 'No', None, None, 'Yes']
TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]
{0: {'Yes': 0.08000000000000002, 'No': 0.0}, 1: {'Yes': 0.08000000000000002, 'No': 0.0}}
['Yes', 'Yes', None, 'Yes', 'Yes', None, 'No', None, None, 'Yes']
TRAIN: [0 1 2 3 4 5 6 9] TEST: [7 8]
{7: {'Yes': 0.03125, 'No': 0.03125}, 8: {'Yes': 0.03125, 'No': 0.03125}}
['Yes', 'Yes', None, 'Yes', 'Yes', None, 'No', 'Yes', 'Yes', 'Yes']
TRAIN: [0 1 3 4 6 7 8 9] TEST: [2 5]
{2: {'Yes': 0.08000000000000002, 'No': 0.05555555555555555}, 5: {'Yes': 0.045, 'No': 0.027777777777777776}}
['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes']
50.0
